## Importing the data and defining some variables

In [1]:
import cv2
import numpy as np
import os 
from random import shuffle
from tqdm import tqdm

TRAIN_DIR = r'C:\Users\52551\Desktop\catdoghw\train'
TEST_DIR = r'C:\Users\52551\Desktop\catdoghw\test\test'
IMG_SIZE = 50
LR = 1e-3 

MODEL_NAME = 'dogvscats--{}--{}.model'.format(LR,'conv-basic')

## Labelling images

In [2]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat':return [1,0]
    elif word_label == 'dog':return [0,1]

## Creating training data

In [3]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data                     
        

## Some image processing

In [4]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img),img_num])
    np.save('test_data.npy', testing_data)
    return testing_data

## Creating or loading training data

In [5]:
train_data = create_train_data()
#if you already have train data:
#train_data = np.load('train_data.npy', allow_pickle=True)

100%|████████████████████████████████████████████████████████████████| 25000/25000 [00:24<00:00, 1018.62it/s]
C:\Users\52551\anaconda3\envs\tf\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


## Define the model (we ignore the warnings)

In [6]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# if the model already exists, just load it
#if os.path.exists('{}.meta'.format(MODEL_NAME)):
   # model.load(MODEL_NAME)
    #print('model loaded!')

## Data segmentation

In [8]:
train = train_data[:-500]
test = train_data[-500:]

In [9]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]


## Saving the arrays with pickle so we dont process everytime


import pickle
X = np.array(X)
y = np.array(Y)
pickle.dump(X, open('X.pkl','wb'))
pickle.dump(y, open('y.pkl','wb'))


## Training that model!!

In [10]:
model.fit({'input': X}, {'targets': Y}, n_epoch=3, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 1148  | total loss: 0.45274 | time: 20.285s
| Adam | epoch: 003 | loss: 0.45274 - acc: 0.7808 -- iter: 24448/24500
Training Step: 1149  | total loss: 0.46380 | time: 21.339s
| Adam | epoch: 003 | loss: 0.46380 - acc: 0.7730 | val_loss: 0.45879 - val_acc: 0.7780 -- iter: 24500/24500
--


In [11]:
#model.save(MODEL_NAME)